# Word embedding models

In [1]:
from ipynb.fs.full.koselleck import *

In [2]:
# get_veclib_word?

## Skipgrams

### Generated elsewhere...

### Loading

In [3]:
def get_skipgrams(idir=PATH_SKIPGRAMS_YR,skipgram_n=25, calc_numlines=False):
    odf=pd.DataFrame([
        {
            'corpus':fn.split('.')[2],
            'year':int([x for x in fn.split('.') if x.isdigit()][0]),
#             'period_end':int([x for x in fn.split('.') if x.isdigit()][-1]),
            'path':os.path.join(idir,fn)
        }
        for fn in os.listdir(idir)
        if fn.startswith('data.skipgrams')
    ]).sort_values(['corpus','year'])
    if calc_numlines:
        odf['num_lines']=odf.path.progress_apply(get_numlines)
        odf['num_words']=odf['num_lines']*skipgram_n
    return odf#.query('1680<=year<1970')

In [4]:
# get_skipgrams()

In [5]:
dfskip=get_skipgrams(calc_numlines=False)
dfskip['period']=dfskip.year.apply(lambda y: periodize_bystep(y,40,1700))
dfskip

,corpus,year,path,period
182,bpo,1681,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1681.txt,
56,bpo,1682,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1682.txt,
51,bpo,1683,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1683.txt,
60,bpo,1684,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1684.txt,
200,bpo,1685,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1685.txt,
...,...,...,...,...
116,bpo,1967,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1967.txt,1940-1980
20,bpo,1968,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1968.txt,1940-1980
34,bpo,1969,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1969.txt,1940-1980
87,bpo,1970,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1970.txt,1940-1980


In [7]:
# dfskip.groupby('period').num_words.sum()

In [8]:
def get_dfskipruns(dfskip,num_runs=10,incl_existing=False):
    dfskipruns=pd.concat([
        dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
        for i in range(num_runs)
    ])
    dfskipruns['opath']=dfskipruns.apply(lambda row: os.path.join(PATH_MODELS,row.corpus,row.period,row.run,'model.bin'),1)
    dfskipruns['opath_exists']=dfskipruns.opath.apply(lambda x: os.path.exists(x))
    if not incl_existing: dfskipruns=dfskipruns[dfskipruns.opath_exists==False]
    return dfskipruns

In [9]:
# get_dfskipruns(dfskip,num_runs=2)

## Generate models

In [10]:

class SkipgramsSampler2:
    def __init__(self,fn,num_skips_wanted=100000,replace=True):
        self.fn=fn
        self.num_skips_wanted=num_skips_wanted
        if os.path.exists(fn):
            with xopen(fn) as f:
                lines=[ln.strip().split() for ln in f if ln.strip()]
        if not replace and len(lines)<num_skips_wanted:
            num_skips_wanted=len(lines)
        self.skips=random.choices(lines,k=num_skips_wanted)
    def __iter__(self):
        yield from self.skips
    def __len__(self): return len(self.skips)



class SkipgramsSampler:
    def __init__(self, fn, num_skips_wanted=None, replace=False):
            self.fn=fn
            self.replace=replace
            
            if num_skips_wanted:                
                self.num_skips=self.get_num_lines()
                self.num_skips_wanted=nskip=num_skips_wanted if self.num_skips>num_skips_wanted else self.num_skips
                self.line_nums_wanted = set(random.sample(list(range(nskip)), nskip))
            else:
                self.num_skips_wanted=None

    def get_num_lines(self):
            then=time.time()
#               print('>> [SkipgramsSampler] counting lines in',self.fn)
            with gzip.open(self.fn,'rb') if self.fn.endswith('.gz') else open(self.fn) as f:
                    for i,line in enumerate(f):
                            pass
            num_lines=i+1
            now=time.time()
#               print('>> [SkipgramsSampler] finished counting lines in',self.fn,'in',int(now-then),'seconds. # lines =',num_lines,'and num skips wanted =',self.num_skips_wanted)
            return num_lines

    def __iter__(self):
            i=0
            with gzip.open(self.fn,'rb') if self.fn.endswith('.gz') else open(self.fn) as f:
                    for i,line in enumerate(f):
                            line = line.decode('utf-8') if self.fn.endswith('.gz') else line
                            if not self.num_skips_wanted or i in self.line_nums_wanted:
                                    yield line.strip().split()

In [18]:
class SkipgramsSamplers:
    def __init__(self,fns,nskip,sampler=SkipgramsSampler,**y):
        self.skippers=[sampler(fn,nskip) for fn in fns]
    def __iter__(self):
        for skipper in self.skippers:
            yield from skipper

In [21]:
# skipfn=dfskip.iloc[0].path
# iterr=SkipgramsSampler(skipfn,None)
# l=[]
# for x in tqdm(iterr): l+=[x]
# print(random.choice(l))
# print(len(l))

In [ ]:
DEFAULT_NUM_SKIP=None

In [21]:
def gen_and_save_model(dfskip,nskip=DEFAULT_NUM_SKIP,force=False,vector_size=100,window=10,min_count=5,epochs=10,workers=8,verbose=False):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        disable_gensim_logging() if not verbose else enable_gensim_logging()
        model = Word2Vec(sentences=ss,vector_size=vector_size,window=window,min_count=min_count,epochs=epochs,workers=workers)
        model.save(ofnfn)
    return pd.DataFrame([{'fnfn':ofnfn}])


In [22]:
# fnfn=gen_and_save_model(get_dfskipruns(dfskip).iloc[:1], force=True).fnfn.iloc[0]
# load_model(fnfn).wv.most_similar('value')

In [26]:
# res=pmap_groups(
#     gen_and_save_model,
#     dfskipruns.groupby(['period','run']),
#     num_proc=4,
#     kwargs=dict(force=True, nskip=NSKIP_PER_YR)
# )

In [22]:
# periodize(1801,40,1700)

In [36]:
def gen_models(
        ybin=5,
        ymin=1680,
        ymax=1970,
        num_runs=1,
        force=False,
        nskip_per_yr=NSKIP_PER_YR
    ):
    dfskip=get_skipgrams(calc_numlines=False).query(f'{ymin}<=year<{ymax}')
    dfskip['period']=dfskip.year.apply(lambda y: periodize_bystep(y,ybin,ymin))
    dfskip=dfskip[dfskip.period!='']
    dfskipruns=get_dfskipruns(dfskip, num_runs=num_runs, incl_existing=force)
    dfgrps=dfskipruns.groupby(['period','run'])
    print(f'Generating {len(dfgrps)} new models over {dfskipruns.period.nunique()} periods and {dfskipruns.run.nunique()} runs')
    return pmap_groups(
        gen_and_save_model,
        dfskipruns.groupby(['period','run']),
        num_proc=5,
        kwargs=dict(force=force, nskip=nskip_per_yr)
    )

In [37]:
gen_models(ybin=40,ymin=1700,ymax=1900,num_runs=1,nskip_per_yr=None)

stop

[Koselleck] (10:24:05) Generating 5 new models over 5 periods and 1 runs (+34.5s)


,corpus,year,path,period,run,opath,opath_exists
157,bpo,1700,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1700.txt,1700-1740,run_01,/home/ryan/github/koselleck/data1/models/bpo/1700-1740/run_01/model.bin,False
42,bpo,1701,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1701.txt,1700-1740,run_01,/home/ryan/github/koselleck/data1/models/bpo/1700-1740/run_01/model.bin,False
25,bpo,1702,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1702.txt,1700-1740,run_01,/home/ryan/github/koselleck/data1/models/bpo/1700-1740/run_01/model.bin,False
288,bpo,1703,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1703.txt,1700-1740,run_01,/home/ryan/github/koselleck/data1/models/bpo/1700-1740/run_01/model.bin,False
69,bpo,1704,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1704.txt,1700-1740,run_01,/home/ryan/github/koselleck/data1/models/bpo/1700-1740/run_01/model.bin,False
...,...,...,...,...,...,...,...
21,bpo,1895,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1895.txt,1860-1900,run_01,/home/ryan/github/koselleck/data1/models/bpo/1860-1900/run_01/model.bin,False
108,bpo,1896,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1896.txt,1860-1900,run_01,/home/ryan/github/koselleck/data1/models/bpo/1860-1900/run_01/model.bin,False
17,bpo,1897,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1897.txt,1860-1900,run_01,/home/ryan/github/koselleck/data1/models/bpo/1860-1900/run_01/model.bin,False
285,bpo,1898,/home/ryan/github/koselleck/data1/skipgrams/years/data.skipgrams.bpo.1898.txt,1860-1900,run_01,/home/ryan/github/koselleck/data1/models/bpo/1860-1900/run_01/model.bin,False


In [15]:
# gen_models(num_runs=10)

## Test models

### Getting model paths

In [16]:

            
def get_model_paths(model_dir=PATH_MODELS,model_fn='model.bin',vocab_fn='vocab.txt',period_len=None):
    """
    Get all models' paths
    """
    ld=[]
    for root,dirs,fns in tqdm(os.walk(model_dir),desc='Scanning directory for models'):
        if model_fn in fns:
            corpus,period,run=root.split('/')[-3:]
            if not 'run_' in run:
                corpus,period=root.split('/')[-2:]
                run=None
            dx={
                'corpus':corpus,
                'period_start':int(period.split('-')[0]),
                'period_end':int(period.split('-')[-1]),
                'path':os.path.join(root,model_fn),
                'path_vocab':os.path.join(root,vocab_fn)
            }
            if run is not None: dx['run']=run
            if period_len and int(dx['period_end'])-int(dx['period_start'])!=period_len:
                continue
            ld.append(dx)
    return ld

In [17]:
YMIN,YMAX,YEARBIN

(1720, 1900, 5)

In [18]:
def get_pathdf_models(period_len=YEARBIN,ymin=YMIN,ymax=YMAX):
    pathdf=pd.DataFrame(get_model_paths(PATH_MODELS_BPO, 'model.bin'))#.sort_values(['period_start','run'])
    pathdf['period']=[f'{x}-{y}' for x,y in zip(pathdf.period_start, pathdf.period_end)]
    pathdf['period_len']=pathdf.period_end - pathdf.period_start
    pathdf['qstr']=[
        f'vecs({period}_{run.split("_")[-1]})'
        for period,run in zip(pathdf.period, pathdf.run)
    ]
    if period_len: pathdf=pathdf[pathdf.period_len==period_len]
    if ymin: pathdf=pathdf[pathdf.period_start>=ymin]
    if ymax: pathdf=pathdf[pathdf.period_end<=ymax]
    return pathdf[~pathdf.period.isnull()].sort_values('period_start')

In [19]:
# get_pathdf_models()

In [20]:
def get_default_models(ymin=YMIN,ymax=YMAX,ybin=YEARBIN,num_runs=10):
    if os.path.exists(FN_DEFAULT_MODEL_PATHS):
        odf=read_df(FN_DEFAULT_MODEL_PATHS)
    else:
        odf=get_pathdf_models(period_len=ybin)
        odf.to_pickle(FN_DEFAULT_MODEL_PATHS)
    return odf.query(f'{ymin}<=period_start & period_end<={ymax} & run<="run_{num_runs:02}"')
    

def get_default_periods(**y):
    return sorted(list(set(get_default_models(**y).period)))

In [1]:
# get_default_periods()

## Input prep

In [22]:
def get_periods_runs(period_or_periods=None,run_or_runs=None,num_runs=10):
    periods=period_or_periods
    if periods is None: periods=get_default_periods()
    if type(periods)==str: periods=tokenize_fast(periods)
    periods=set(periods)
    runs=run_or_runs    
    if runs is None: runs=list(range(1,num_runs+1))
    if type(runs)==int: runs=[runs]
    if type(runs)==str: runs=[int(runs)]
    runs=set(runs)
    return periods,runs

In [2]:
# get_periods_runs('1770-1775,1780-1785')

In [24]:
# get_default_models()

In [1]:
# get_default_periods(ymin=1800)

### Loading models

In [4]:

def load_model(path_model,path_vocab=None,min_count=None,cache_bin=True,cache=False):
    global MODEL_CACHE
    
    if cache and path_model in MODEL_CACHE: return MODEL_CACHE[path_model]
#     print('Loading',path_model)
    model=do_load_model(path_model,path_vocab=path_vocab,min_count=min_count,cache_bin=cache_bin)
    return model
    
def do_load_model(path_model,path_vocab=None,min_count=None,cache_bin=True):
#     print('>> loading',path_model)
    path_model_bin=path_model.split('.txt')[0]+'.bin' if not path_model.endswith('.bin') else path_model
    if os.path.exists(path_model_bin):
        model=gensim.models.KeyedVectors.load(path_model_bin,mmap='r')
    elif os.path.exists(path_model):
        if not path_vocab: path_vocab=os.path.join(os.path.dirname(path_model,'vocab.txt'))
        if os.path.exists(path_vocab):
            model = gensim.models.KeyedVectors.load_word2vec_format(path_model,path_vocab)
            if min_count: filter_model(model,min_count=min_count)
        else:
            model = gensim.models.KeyedVectors.load_word2vec_format(path_model)
        if cache_bin:
            model.save(path_model_bin)
    else:
        print('!!??',path_model)
        stop
        return None
#     print(path_model, len(model.wv.key_to_index))
    return model
    

In [5]:
# row = get_default_models().sample(n=1).iloc[0]
# load_model_row(row)

In [42]:
# m=load_model('/home/ryan/github/koselleck/data1/models/bpo/1805-1810/run_25/model.bin')
# m.wv.most_similar('virtue')

In [41]:
# m=load_model('/home/ryan/github/koselleck/data1/models/bpo/1945-1950/run_07/model.bin')
# m.wv.most_similar(['king','woman'],['man'])

In [30]:
def test_models(dfmodels,gby=['period','run']):
    o=[]
    dfgrp=dfmodels.groupby(gby)
    for period,dfg in tqdm(sorted(dfgrp)):#, total=len(dfgrp)):
        path=dfg.iloc[-1].path
        m=load_model(path)
        try:
            testvec=m.wv.most_similar(['king','woman'],['man'],topn=25)
        except KeyError:
            continue
        testvec_wl=[x for x,y in testvec]
        has_queen='queen' in set(testvec_wl)
        odx={
            **dict(zip(gby,period)),
            'has_queen':has_queen,
            'rank_queen':testvec_wl.index('queen') if has_queen else np.nan,
            'neighborhood':', '.join(testvec_wl),
        }
        o+=[odx]
#         break
    return pd.DataFrame(o)

In [31]:
# dfmodels = get_pathdf_models().query('period_len==5')
# dftests  = test_models(dfmodels)
# dftests.to_csv('../../data/data.model.tests.csv')
# dftests.query('has_queen==True').groupby('period').size()
# dftests

In [32]:
def get_new_veclib_word_data_path(word):
    ofn=os.path.join(PATH_DB,'cdists',f'data.cdists.{word}.pkl.gz')
    odir=os.path.dirname(ofn)
    if not os.path.exists(odir): os.makedirs(odir)
    return ofn

In [33]:
get_new_veclib_word_data_path('virtue')

'/home/ryan/github/koselleck/db/cdists/data.cdists.virtue.pkl.gz'

In [34]:
def get_veclib_word_data(word,progress=True,cache=True,cache_only=False,force=False,remove_old=True,
                        periods=None):
    if progress: print(f'Loading cdist data for "{word}"')
    odf=pd.DataFrame()
    fnfn=get_new_veclib_word_data_path(word)
    oldfnfn=get_old_veclib_word_data_path(word)
    if cache and not force and os.path.exists(fnfn):
        try:
            odf=read_df(fnfn)
            if progress: print(f'Finished loading cdist data from pkl for "{word}"')
        except Exception as e:
            print('!!',e)
    if not len(odf):
        if not os.path.exists(oldfnfn):
            if progress: print(f'No file found at {oldfnfn}')
        else:
            with get_veclib_word(word) as vl:
                dfdist=pd.DataFrame(dict(vl.items())).T.rename_axis('period_run_')
                dfdist['period_'],dfdist['run_']=zip(*[x.split('_') for x in dfdist.index])
                dfdist['run_']=dfdist['run_'].apply(int)
                odf=dfdist.reset_index().drop('period_run_',1).set_index(['period_','run_'])
                if cache:
                    if progress: print(f'Saving dfdist to "{fnfn}"')
                    odf.to_pickle(fnfn)
                    if remove_old and os.path.exists(oldfnfn):
                        if progress: print(f'Removing old data from "{oldfnfn}"')
                        os.remove(oldfnfn)
                if progress: print(f'Finished loading cdist data from sqlite for "{word}"')
    if not len(odf): return odf
    odf['word_']=word
    odf=odf.reset_index()
    
    if periods is None: periods=set(get_default_periods())
    odf=odf[odf.period_.isin(periods)]
    odf=odf.set_index(['word_','period_','run_']).rename_axis(['word','period','run'])
    return odf

In [35]:
# dfdist=get_veclib_word_data('histories',force=False,remove_old=False,cache_only=False)
# dfdist

In [36]:
def get_all_words_in_sqlite_data():
    fns=os.listdir(os.path.join(PATH_DB,'wvecs'))
    words=[fn.split('.sqlite')[0].split('.')[-1] for fn in fns]
    return words

In [37]:
# words=get_all_words_in_sqlite_data()
# len(words),random.sample(words,10)

In [38]:
def _get_veclib_word_data_(objd): return get_veclib_word_data(**objd)
def reformat_all_sqlite_data(words=None,lim=None,num_proc=1,remove_old=True):
    words=get_all_words_in_sqlite_data()[:lim] if words is None else list(words)[:lim]
    return pmap(
        _get_veclib_word_data_,
        [dict(word=word,progress=False,cache=True,force=True,remove_old=remove_old,
              cache_only=True) for word in words],
        num_proc=num_proc,
        desc='Reformatting old sqlite data into pkl files',
        use_threads=False
    )

In [39]:
# words=get_valid_words()
# random.shuffle(words)
# res=reformat_all_sqlite_data(words,lim=None,num_proc=4)

## Misc. functions

In [40]:


def measure_ambiguity(model,words=None,topn=10):
    dfdist=to_dist(model,words=words)
    g=to_semnet_from_dist(dfdist,topn=topn)
    s=pd.Series(nx.clustering(g)).sort_values()
    amb=1-s
    return amb

def to_dist(m,words=None,z=1,norm=1,maxwords=10000):
    if words is None:
        words=[m.wv.index_to_key[i] for i in range(maxwords)]
    else:
        words=set(words) & set(m.wv.key_to_index.keys())
    words=list(words)
    vecs = np.array([m.wv[w] for w in words], dtype=np.float64)
    omatrix = fastdist.cosine_pairwise_distance(vecs, return_matrix=True)
    odf = round(pd.DataFrame(omatrix, index=words, columns=words),6)
    maxv=odf.max().max()
    odf=odf.replace({maxv:np.nan})
    if norm: odf=maxv - odf
    if z: odf=(odf - odf.mean().mean())/odf.std().std()
    return odf

def to_semnet_from_dist(dfdist,cutoff=3,topn=10):
    g=nx.Graph()
    for word1 in dfdist.columns:
        for word2,val in dfdist[word1].sort_values(ascending=True).iloc[:topn].items():
            g.add_edge(word1,word2,weight=val*-1)
    return g

def measure_ambiguity(model,words=None,topn=10,z=False):
    dfdist=to_dist(model,words=words)
    g=to_semnet_from_dist(dfdist,topn=topn)
    s=pd.Series(nx.clustering(g)).sort_values()
    amb=1-s
    if z: amb=(amb - amb.mean())/amb.std()
    return amb

def get_any_model(dfpath=None):
    if dfpath is None: dfpath=get_pathdf_models_bydecade()
    row=dfpath.sample(n=1).iloc[0]
    return load_model_row(row)

def measure_freq(model,words=None,tf=True,z=False):
    mwords=set(model.wv.key_to_index.keys())
    words=mwords if not words else mwords&set(words)
    vocabd=dict(
        (
            w,
            model.wv.get_vecattr(w,'count')
        )
        for w in words
    )
    svocab=pd.Series(vocabd)
    if tf: svocab=svocab / svocab.sum()
    if z: svocab=(svocab - svocab.mean()) / svocab.std()
    return svocab

INFLECTER=None
def measure_singularism(m,words=None,z=True):
    global INFLECTER
    if INFLECTER is None:
        import inflect
        INFLECTER=inflect.engine() 
    p=INFLECTER

    if not words: words=get_valid_words()
    words=list(set(words) & set(m.wv.key_to_index.keys()))
    words_plurals = pmap(p.plural_noun, words, num_proc=1, progress=False)
    s=measure_freq(m,words=set(words+words_plurals),z=z)
    sd=dict(s)
    odf=pd.DataFrame([
        {'word':ws, 'word_pural':wp, 'freq_sing':sd.get(ws), 'freq_plural':sd.get(wp)}
        for ws,wp in zip(words,words_plurals)
        if ws!=wp
    ])
    odf['freq_diff']=odf['freq_sing']-odf['freq_plural']
    odf['rank_sing']=odf['freq_sing'].rank(ascending=True)
    odf['rank_plural']=odf['freq_plural'].rank(ascending=True)
    odf['rank_diff']=odf['rank_sing']-odf['rank_plural']
    if z:
        for x in odf.select_dtypes('number').columns:
            odf[x]=(odf[x] - odf[x].mean())/odf[x].std()
    return odf.set_index('word').sort_values('rank_diff').dropna()


In [ ]:
def smart_procrustes_align_gensim(base_embed, other_embed, words=None, combine_models=False):
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # patch by Richard So [https://twitter.com/richardjeanso) (thanks!) to update this code for new version of gensim
    # base_embed.init_sims(replace=True)
    # other_embed.init_sims(replace=True)

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)

    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    other_vecs = in_other_embed.wv.get_normed_vectors()

    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 
    # SVD method from numpy
    u, _, v = np.linalg.svd(m)
    # another matrix operation
    ortho = u.dot(v) 
    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    
    if combine_models:
        o=union_align_gensim(base_embed,other_embed)
    else:
        o=other_embed
    return o
    

def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
    return (m1,m2)


def union_align_gensim(m1, m2, words=None, suffix1='_m1',suffix2='_m2'):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """

    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))

    # Then for each model...
    new_arr_both=[]
    common_vocab_both=[w+suffix1 for w in common_vocab] + [w+suffix2 for w in common_vocab]
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = [old_arr[index] for index in indices]
        new_arr_both+=new_arr
    
#     odf=pd.DataFrame(new_arr_both,index=common_vocab_both)
#     return odf
    
    m.wv.vectors = np.array(new_arr_both)

    # Replace old vocab dictionary with new one (with common vocab)
    # and old index2word with new one
    new_key_to_index = {}
    new_index_to_key = []
    for new_index, key in enumerate(common_vocab_both):
        new_key_to_index[key] = new_index
        new_index_to_key.append(key)
    m.wv.key_to_index = new_key_to_index
    m.wv.index_to_key = new_index_to_key
    return m